In [ ]:
import os
import sys
import glob
import numpy as np
# from scipy.fft import fft, fftfreq
from scipy.signal import welch
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})
OU_tau = 20e-3

In [ ]:
# expt_name = 'G02_double_H_const_P_loads'
expt_name = os.path.join('default_const_P_loads','1_load')
steps_per_decade = 1000
folder = os.path.join('..','..','modal_analysis','IEEE39',expt_name)
AC_data_file = os.path.join(folder, f'IEEE39_AC_TF_-6.0_2.0_{steps_per_decade}.npz')
AC_data = np.load(AC_data_file, allow_pickle=True)
dB = AC_data['dB'].item()

In [ ]:
if False:
    AC_data_2 = [np.load(f, allow_pickle=True) for f in sorted(glob.glob(os.path.join(
        '..','..','modal_analysis','IEEE39','default',f'IEEE39_AC_TF_-6.0_2.0_{steps_per_decade}_Load_*.npz')))]

In [ ]:
discard = 300 # [s]
tran_data_files = sorted(glob.glob(os.path.join(folder,'IEEE39_tran_*.npz')))
# tran_data_files = tran_data_files[:3]
print(f'Found {len(tran_data_files)} data files.')
tran_blobs = [np.load(f, allow_pickle=True) for f in tran_data_files]
time = [blob['time'] for blob in tran_blobs]
speed = [blob['data'].item()['gen']['s:xspeed'] for blob in tran_blobs]
tran_speed = np.concatenate([spd[t>discard,:] for t,spd in zip(time, speed)], axis=0)
dt = time[0][1] - time[0][0]
tran_time = np.arange(tran_speed.shape[0]) * dt

In [ ]:
tran_M = tran_blobs[0]['momentum'].item()
AC_M = AC_data['Mtot'].item()
M = tran_M

In [ ]:
fref = 1
Δω = (tran_speed - 1) * fref
window_dur = 60 * 15
window = window_dur / dt
onesided = True
tran_freq,tran_Pxx = welch(Δω, 1/dt, window='boxcar', nperseg=window, noverlap=window/2,
                           nfft=window, return_onesided=True, scaling='density', axis=0)
tran_Pxx /= 2
tran_freq,tran_Pxx = tran_freq[1:],tran_Pxx[1:,:]

In [ ]:
device_names = tran_blobs[0]['device_names'].item()['gen']
if len(device_names) == 10:
    rows,cols = 2,5
elif len(device_names) == 18:
    rows,cols = 3,6
w,h = 3,2.5
fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h), sharex=True, sharey=True)
ticks = np.logspace(-3, 2, 6)
F0 = 0.1
red,green,magenta,orange = [0.75,0,0], [0,.75,0], [.75,0,.75], [1,.5,0]
col = [green,magenta]
cmap = plt.get_cmap('tab10', len(AC_data))
lw = 1
# names = tran_blobs[0]['device_names'].item()['gen']
AC_freq = AC_data['F']
coeff = 0
for k,name in enumerate(device_names[:rows*cols]):
    i,j = k//cols, k%cols
    try:
        ### 1
        idx = device_names.index(name)
        jdx = np.abs(tran_freq-F0).argmin()
        y = 10 * np.log10(tran_Pxx[:,idx])
        y -= coeff*y[jdx]
        ym,yM = y.min(),y.max()
        ax[i,j].semilogx(tran_freq, y, 'k', lw=lw, label='TRAN: M = {:.0f} MJs'.format(M))
        ### 2
        idx = np.where(AC_data['SM_names'] == name)[0][0]
        jdx = np.abs(AC_freq-F0).argmin()
        y = AC_data['mag'][:,idx]
        y -= coeff*y[jdx]
        if y.min() < ym:
            ym = y.min()
        if y.max() > yM:
            yM = y.max()
        ax[i,j].semilogx(AC_freq, y, color=red, lw=1, label='AC')
        ### 3
#         try:
#             for n,data in enumerate(AC_data_2):
#                 y = data['mag'][:,idx]
#                 ax[i,j].semilogx(AC_freq, y-coeff*y[jdx], color=col[n], lw=1, label=f'AC load {n+1}')
#         except:
#             pass
        ### 4
        ax[i,j].plot(1/(2*np.pi*OU_tau)+np.zeros(2), [ym,yM], ':', color=orange, lw=2, label='OU cutoff')
        ax[i,j].set_title(name.split('___')[0], fontsize=8)
        ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
        ax[i,j].xaxis.set_minor_locator(NullLocator())
        ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
        ax[i,j].set_xlim(ticks[[0,-1]])
    except:
        print(f'Device name {name} missing')
ax[0,0].legend(loc='lower left', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
for a in ax[:,0]:
    a.set_ylabel(f'PSD [dB{dB}]')
sns.despine()
fig.tight_layout()
plt.savefig(os.path.join(folder, 'spectra_comparison.pdf'))